In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
## you can comment the following 2 lines if you run the code in Jupyter Notebook/Lab
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_row", None)

"""
Author: Yixian Zhou, ...
Date: Nov. 9
Description: Exploaration. Data Cleaning: extrated state names from the GeoName and added some columns
"""

####### SET UP #######
STATE_ABBR = """Alabama - AL
Alaska - AK
Arizona - AZ
Arkansas - AR
California - CA
Colorado - CO
Connecticut - CT
District of Columbia - DC
Delaware - DE
Florida - FL
Georgia - GA
Hawaii - HI
Idaho - ID
Illinois - IL
Indiana - IN
Iowa - IA
Kansas - KS
Kentucky - KY
Louisiana - LA
Maine - ME
Maryland - MD
Massachusetts - MA
Michigan - MI
Minnesota - MN
Mississippi - MS
Missouri - MO
Montana - MT
Nebraska - NE
Nevada - NV
New Hampshire - NH
New Jersey - NJ
New Mexico - NM
New York - NY
North Carolina - NC
North Dakota - ND
Ohio - OH
Oklahoma - OK
Oregon - OR
Pennsylvania - PA
Rhode Island - RI
South Carolina - SC
South Dakota - SD
Tennessee - TN
Texas - TX
Utah - UT
Vermont - VT
Virginia - VA
Washington - WA
West Virginia - WV
Wisconsin - WI
Wyoming - WY"""
MAX_NUM_STATE = 4
DATA_PATH = "./dataset"
FILENAME = "./dataset/climate_dataset_cleaned_v1.0.csv"
NUM_STATES = 51 ## well... it should be 50 but they included District of Columbia
def is_multi_state(s):
    """return 1 if the county is a multi-state area"""
    if "-" in s:
        return 1
    else:
        return 0

def get_num_states(s):
    """return the number of states"""
    if ("US" in s):
        return NUM_STATES
    _state_list = s.split('-')
    return len(_state_list)

def convert2full(s, abbr2full):
    """convert abbreviations to full names"""
    if ("US" in s) | (len(s) > 2) & ("-" not in s) : # record for country # record for states
        return s        
    elif "-" in s:
        return "-".join([abbr2full[abbr] for abbr in s.split("-")])
    else:
        return abbr2full[s]

####### LOAD DATA AND DATA CLEANING #######
climate = pd.read_csv(f"{DATA_PATH}/YCOM_2016_Data.01.csv")
metadata = pd.read_csv(f"{DATA_PATH}/YCOM_Metadata_2017Feb20.csv")

raw_cols = list(climate.columns)
pair_list = STATE_ABBR.split('\n')
abbr2full = {s.split('-')[1].strip(): s.split('-')[0].strip() for s in pair_list} # abbr_dict = {}

climate['StateName_raw'] = climate['GeoName'].str.split(", ").str[-1]
climate['StateName'] = climate['StateName_raw'].apply(lambda x: convert2full(x, abbr2full))
climate['GeoName_new'] = climate['GeoName'].str.split(", ").str[0]
# climate = climate[np.insert(climate.columns, 3, "GeoName_new")[:-1]] move columns name
climate['IsMultiState'] = climate['StateName_raw'].apply(lambda x: is_multi_state(x))
climate['NumStates'] = climate['StateName_raw'].apply(lambda x: get_num_states(x))

for i in range(MAX_NUM_STATE):
    climate[f'StateName_{i + 1}'] = climate['StateName'].str.split('-').str[i].str.strip()
    climate[f'StateName_{i + 1}'] = climate[f'StateName_{i + 1}'].fillna("Empty")

climate = climate[raw_cols[:3] + ['GeoName_new', 'StateName_raw', 'StateName', 'IsMultiState', 'NumStates'] + \
                  [f"StateName_{i + 1}" for i in range(MAX_NUM_STATE)] + raw_cols[3:]]

climate.to_csv(FILENAME, index=False)

In [32]:
###### SOMETHINE ELSE ######
print("Total Pop...")
print("Total population of all the counties in Alabama:", climate[(climate['StateName'].str.contains("\w*Alabama\w*")) & (climate['GeoType'] != 'State')]['TotalPop'].sum())
print("Total population in Alabama:", climate[(climate['StateName'] == "Alabama") & (climate['GeoType'] == 'State')]['TotalPop'].sum())
print("Different GeoType")
print(climate['GeoType'].value_counts())

Total Pop...
Total population of all the counties in Alabama: 16553682
Total population in Alabama: 3217902
Different GeoType
County      3142
CBSA         916
cd113        435
State         51
National       1
Name: GeoType, dtype: int64


In [3]:
# climate[climate['GeoType'] == "cd113"]

In [4]:
print(f"Number of States: {len(climate[climate['GeoType'] == 'State'])}")
print("List of States in the dataset:\n", list(climate[climate['GeoType'] == "State"]["GeoName"]))

Number of States: 51
List of States in the dataset:
 ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [7]:
print("All the columns:\n", list(climate.columns))

All the columns:
 ['GeoType', 'GEOID', 'GeoName', 'GeoName_new', 'StateName_raw', 'StateName', 'IsMultiState', 'NumStates', 'StateName_1', 'StateName_2', 'StateName_3', 'StateName_4', 'TotalPop', 'discuss', 'discussOppose', 'CO2limits', 'CO2limitsOppose', 'trustclimsciSST', 'trustclimsciSSTOppose', 'regulate', 'regulateOppose', 'supportRPS', 'supportRPSOppose', 'fundrenewables', 'fundrenewablesOppose', 'mediaweekly', 'mediaweeklyOppose', 'happening', 'happeningOppose', 'human', 'humanOppose', 'consensus', 'consensusOppose', 'worried', 'worriedOppose', 'personal', 'personalOppose', 'harmUS', 'harmUSOppose', 'devharm', 'devharmOppose', 'futuregen', 'futuregenOppose', 'harmplants', 'harmplantsOppose', 'timing', 'timingOppose']


In [ ]:
# climate.describe()

In [19]:
# print("======= what is in the metadata =======")
# for i in metadata.index:
#     print(" ",metadata.iloc[i]['YCOM VARIABLE NAME'], "\n",metadata.iloc[i]['VARIABLE DESCRIPTION'])

  Statecode 
 Geographic abbreviation
  Statename 
 Geographic name
  TotalPop 
 Total population
  CO2limits 
 Estimated percentage who somewhat/strongly support setting strict limits on existing coal-fire power plants
  CO2limitsOppose 
 Estimated percentage who somewhat/strongly oppose setting strict limits on existing coal-fire power plants
  regulate 
 Estimated percentage who somewhat/strongly support regulating CO2 as a pollutant
  regulateOppose 
 Estimated percentage who somewhat/strongly oppose regulating CO2 as a pollutant
  supportRPS 
 Estimated percentage who somewhat/strongly support requiring utilities to produce 20% electricity from renewable sources
  supportRPSOppose 
 Estimated percentage who somewhat/strongly oppose requiring utilities to produce 20% electricity from renewable sources
  fundrenewables 
 Estimated percentage who somewhat/strongly support funding research into renewable energy sources
  fundrenewablesOppose 
 Estimated percentage who somewhat/strongl